This notebook develops data transformations and formatting to prepare for use in a model. Code is intended to be functional and able to be copied to new notebooks for submission or other use.

## Package imports:

In [1]:
import gresearch_crypto

from datetime import datetime
import pandas as pd

## Declare variables:

In [2]:
train_filepath = '/kaggle/input/g-research-crypto-forecasting/train.csv'
asset_details_filepath = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

## Import data:

In [3]:
train_df = pd.read_csv(train_filepath)
asset_details_df = pd.read_csv(asset_details_filepath)

In [4]:
train_df.tail()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN
24236805,1632182400,11,48.0,232.695000,232.800000,232.240000,232.275000,1.035123e+02,232.569697,NaN


In [5]:
asset_details_df.tail()

,Asset_ID,Weight,Asset_Name
9,12,2.079442,Stellar
10,3,4.406719,Cardano
11,8,1.098612,IOTA
12,10,1.098612,Maker
13,4,3.555348,Dogecoin


## Define functions for data cleaning / transforms:

In [6]:
def clean_dates(df):
    '''
    Function to clean timestamps of an individual coin 
    by the earliest and latest timestamps observed for that coin.
    Also fills in missing values by the method chosen for the interpolate function.
    
    Inputs: 
        df (pd.DataFrame.GroupBy object) :
            Grouped Dataframe by unique coins. All timestamps must be
            in intervals of 60 seconds.

    Outputs:
        constant_dates_df (pd.DataFrame) :
            Dataframe with timestamps and filled missing values.
    '''
    
    df = df.copy()
    
    dates = range(min(df["timestamp"]), max(df["timestamp"]), 60)
    
    df.set_index("timestamp", inplace = True)
    
    df = df.reindex(dates)
    
    # don't fill missing values at end of dataset, where they do not
    # have ending observations to interpolate with
    df.interpolate(method = "linear", inplace = True, limit_area = "inside")
    
    return df
    
def standardize_data(df):
    '''
    Function to standardize data by creating rows for every timestamp
    and subsetting to only consider when all coins had their first observation made.
    
    Inputs:
        df (pd.DataFrame) :
            Time series data to be standardized
            
    Outputs:
        standard_df (pd.DataFrame) :
            Time series data now standardized
    '''
    
    # deep copy to not alter the original
    df = df.copy()
    
    # fill missing rows / values between coin's start and stop date
    # Note: Missing rows beyond an individual coin's start / stop date are not created,
    # only those between are filled in
    standard_df = df.groupby("Asset_ID").apply(clean_dates).reset_index(level = 0, drop = True)
    
    # reset twice so timestamp is only a column and not also index, 
    # this makes each entry have a unique index
    standard_df = standard_df.reset_index()
    
    # get the earliest timestamp for each coin, then get the latest timestamp out of those.
    # this shows when the latest coin was introduced, after which there are observations for
    # all coins
    first_timestamp = max(standard_df.groupby("Asset_ID")["timestamp"].min())
    
    # subset to only consider the time period where observations existed for all coins
    standard_df = standard_df.loc[standard_df["timestamp"] >= first_timestamp]
    
    # drop ending rows with missing values
    standard_df.dropna(inplace = True)
    
    return standard_df

## Show problems with dataset that need cleaning:

 - Many missing target values

In [7]:
train_df.isna().sum()

timestamp         0
Asset_ID          0
Count             0
Open              0
High              0
Low               0
Close             0
Volume            0
VWAP              9
Target       750338
dtype: int64

- Different starting dates for each coin

In [8]:
train_df.groupby("Asset_ID")["timestamp"].min()

Asset_ID
0     1514764860
1     1514764860
2     1514764860
3     1523956260
4     1555079640
5     1514764860
6     1514764860
7     1514764860
8     1525853220
9     1514764860
10    1525965660
11    1514764860
12    1518825180
13    1517953020
Name: timestamp, dtype: int64

- All coins do have the same ending dates

In [9]:
train_df.groupby("Asset_ID")["timestamp"].max()

Asset_ID
0     1632182400
1     1632182400
2     1632182400
3     1632182400
4     1632182400
5     1632182400
6     1632182400
7     1632182400
8     1632182400
9     1632182400
10    1632182400
11    1632182400
12    1632182400
13    1632182400
Name: timestamp, dtype: int64

- Related to above, coins have different number of observations

In [10]:
train_df.groupby("Asset_ID").size()

Asset_ID
0     1942619
1     1956282
2     1953537
3     1791867
4     1156866
5     1955140
6     1956200
7     1951127
8     1592071
9     1956030
10     670497
11    1701261
12    1778749
13    1874560
dtype: int64

- Coins have different periods between observations

In [11]:
train_df.groupby("Asset_ID")["timestamp"].diff().value_counts().head()

60.0     23819121
120.0      258885
180.0       75589
240.0       32230
300.0       16674
Name: timestamp, dtype: int64

## Apply cleaning functions to standardize data:

In [12]:
result = standardize_data(train_df)

In [13]:
result

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
671913,1555079640,0.0,13.0,17.840000,17.843600,17.840000,17.840100,9.212400e+02,17.841473,-0.001787
671914,1555079700,0.0,12.0,17.840000,17.856500,17.840000,17.847100,1.270400e+02,17.846861,-0.001602
671915,1555079760,0.0,22.0,17.856500,17.860000,17.850400,17.850400,1.261230e+03,17.857771,-0.000504
671916,1555079820,0.0,5.0,17.859400,17.859500,17.845600,17.859500,2.382000e+01,17.848033,0.000566
671917,1555079880,0.0,4.0,17.853200,17.858300,17.849900,17.849900,3.577000e+02,17.850593,0.000000
...,...,...,...,...,...,...,...,...,...,...
26080009,1632181200,13.0,986.0,0.090885,0.091054,0.090710,0.090941,5.734456e+06,0.090831,0.000199
26080010,1632181260,13.0,715.0,0.091010,0.091277,0.090980,0.091203,1.989048e+06,0.091125,-0.003477
26080011,1632181320,13.0,527.0,0.091171,0.091295,0.091092,0.091180,2.386078e+06,0.091185,-0.002437
26080012,1632181380,13.0,463.0,0.091206,0.091290,0.091028,0.091081,1.568854e+06,0.091131,0.004843


Note that early timestamps are missing for all coins because of the subset.

In [14]:
# show first timestamp
result.loc[result["timestamp"] == 1514764860]

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target


Also note that all coins now have the same number of rows

In [15]:
result.groupby("Asset_ID").size()

Asset_ID
0.0     1285031
1.0     1285031
2.0     1285031
3.0     1285031
4.0     1285031
5.0     1285031
6.0     1285031
7.0     1285031
8.0     1285031
9.0     1285031
10.0    1285031
11.0    1285031
12.0    1285031
13.0    1285031
dtype: int64

And that there is now even spacing between each coin's entries

In [16]:
result.groupby("Asset_ID")["timestamp"].diff().value_counts().head()

60.0    17990420
Name: timestamp, dtype: int64

## Make train / test split

For the purpose of modeling, a meaningful train / test split must be created to ensure models are chosen under conditions close to the evaluation period. The evaluation period is three months long, so a test period of equal length from the training dataset can be used to evaluate models against. 

In [21]:
latest_timestamp = result.loc[:, "timestamp"].max()

# 60 seconds per minute, 60 minutes per hour, 24 hours per day, ~30 days per month,
# for 3 months
cutoff_timestamp = 60*60*24*30*3

standard_train_df = result.loc[(latest_timestamp - result["timestamp"]) > cutoff_timestamp]
standard_test_df = result.loc[(latest_timestamp - result["timestamp"]) <= cutoff_timestamp]

In [22]:
standard_train_df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
671913,1555079640,0.0,13.0,17.840000,17.843600,17.840000,17.840100,9.212400e+02,17.841473,-0.001787
671914,1555079700,0.0,12.0,17.840000,17.856500,17.840000,17.847100,1.270400e+02,17.846861,-0.001602
671915,1555079760,0.0,22.0,17.856500,17.860000,17.850400,17.850400,1.261230e+03,17.857771,-0.000504
671916,1555079820,0.0,5.0,17.859400,17.859500,17.845600,17.859500,2.382000e+01,17.848033,0.000566
671917,1555079880,0.0,4.0,17.853200,17.858300,17.849900,17.849900,3.577000e+02,17.850593,0.000000
...,...,...,...,...,...,...,...,...,...,...
25950408,1624405140,13.0,534.0,0.050454,0.050490,0.050310,0.050389,4.856975e+06,0.050407,-0.000036
25950409,1624405200,13.0,353.0,0.050395,0.050581,0.050277,0.050428,2.997650e+06,0.050405,-0.000741
25950410,1624405260,13.0,336.0,0.050507,0.050598,0.050437,0.050542,1.916351e+06,0.050537,0.000537
25950411,1624405320,13.0,272.0,0.050495,0.050528,0.050400,0.050452,3.344638e+06,0.050467,0.000731


In [23]:
standard_test_df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
1827343,1624405440,0.0,1218.0,259.602000,259.820000,257.414000,257.948000,4.132081e+03,258.484987,0.007666
1827344,1624405500,0.0,1065.0,258.070000,259.990000,257.860000,259.667000,4.787549e+03,259.270848,0.013220
1827345,1624405560,0.0,691.0,259.692000,260.200000,259.354000,259.795000,2.864463e+03,259.719971,0.003928
1827346,1624405620,0.0,911.0,259.745000,260.900000,259.450000,260.845000,2.278637e+03,260.091171,0.002770
1827347,1624405680,0.0,724.0,260.887000,261.250000,260.330000,260.466500,3.134678e+03,260.788116,-0.003693
...,...,...,...,...,...,...,...,...,...,...
26080009,1632181200,13.0,986.0,0.090885,0.091054,0.090710,0.090941,5.734456e+06,0.090831,0.000199
26080010,1632181260,13.0,715.0,0.091010,0.091277,0.090980,0.091203,1.989048e+06,0.091125,-0.003477
26080011,1632181320,13.0,527.0,0.091171,0.091295,0.091092,0.091180,2.386078e+06,0.091185,-0.002437
26080012,1632181380,13.0,463.0,0.091206,0.091290,0.091028,0.091081,1.568854e+06,0.091131,0.004843
